In [0]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings("ignore")

In [0]:
type_name = {
             0: '1_1JHC_0',
             1: '1_1JHC_1',
             2: '2_2JHH',
             3: '3_1JHN',
             4: '4_2JHN',
             5: '5_2JHC',
             6: '6_3JHH',
             7: '7_3JHC',
             8: '8_3JHN', 
            }

In [0]:
type_params = {
    0: {
    'boosting_type': "gbdt",
    'objective': "huber",
    'learning_rate': 0.1,
    'num_leaves': 511,
    'sub_feature': 0.50,
    'sub_row': 0.5,
    'bagging_freq': 1,
    'metric': 'mae'},

    1: {
    'boosting_type': "gbdt",
    'objective': "huber",
    'learning_rate': 0.1,
    'num_leaves': 100,
    'sub_feature': 0.50,
    'sub_row': 0.5,
    'bagging_freq': 1,
    'metric': 'mae'},
  
    #another_sub
    2: {    
    'boosting_type': "gbdt",
    'objective': "huber",
    'learning_rate': 0.01,
    'bagging_freq': 1,
    'metric': 'mae',
    'min_data_in_leaf': 130, 
    'num_leaves': 150, 
    'reg_alpha': 0.5, 
    'reg_lambda': 0.6000000000000001, 
    'sub_feature': 0.30000000000000004, 
    'sub_row': 0.4},
    
#     2: {    
#     'boosting_type': "gbdt",
#     'objective': "huber",
#     'learning_rate': 0.1,
#     'bagging_freq': 1,
#     'metric': 'mae',
#     'min_data_in_leaf': 80, 
#     'num_leaves': 150, 
#     'reg_alpha': 0.30000000000000004, 
#     'reg_lambda': 0.30000000000000004, 
#     'sub_feature': 0.30000000000000004, 
#     'sub_row': 0.60000000000000010},
    
    #another_sub
    3: {
    'boosting_type': "gbdt",
    'objective': "huber",
    'learning_rate': 0.01,
    'bagging_freq': 1,
    'metric': 'mae',
    'min_data_in_leaf': 96, 
    'num_leaves': 30, 
    'reg_alpha': 0.2, 
    'reg_lambda': 0.4, 
    'sub_feature': 0.4, 
    'sub_row': 0.5},
    
#         3: {
#     'boosting_type': "gbdt",
#     'objective': "huber",
#     'learning_rate': 0.01,
#     'bagging_freq': 1,
#     'metric': 'mae',
#     'num_leaves': 50, 
#     'sub_feature': 0.5, 
#     'sub_row': 0.5},
    
    
    4: {
    'boosting_type': "gbdt",
    'objective': "huber",
    'learning_rate': 0.1,
    'bagging_freq': 1,
    'metric': 'mae',
    'min_data_in_leaf': 21, 
    'num_leaves': 200, 
    'reg_alpha': 0.30000000000000004, 
    'reg_lambda': 0.2, 
    'sub_feature': 0.4, 
    'sub_row': 1.0},
    
    5: {
    'boosting_type': "gbdt",
    'objective': "huber",
    'learning_rate': 0.1,
    'bagging_freq': 1,
    'metric': 'mae',
    'num_leaves': 1023, 
    'sub_feature': 0.5, 
    'sub_row': 0.5},
    
    #0.1 opt
    6:{
   'boosting_type': "gbdt",
   'objective': "huber",
   'learning_rate': 0.1,
   'min_data_in_leaf': 50, 
   'num_leaves': 700, 
   'reg_alpha': 0.30000000000000004, 
   'reg_lambda': 0.8, 
   'sub_feature': 0.5, 
   'sub_row': 0.5},
    
    7: {
    'boosting_type': "gbdt",
    'objective': "huber",
    'learning_rate': 0.1,
    'bagging_freq': 1,
    'metric': 'mae',
    'num_leaves': 1023, 
    'sub_feature': 0.5, 
    'sub_row': 0.5},
               

    8: {
    'boosting_type': "gbdt",
    'objective': "huber",
    'learning_rate': 0.01,
    'min_data_in_leaf': 38,
        'num_leaves': 350,
        'reg_alpha': 0.30000000000000004,
        'reg_lambda': 0.6000000000000001,
        'sub_feature': 0.6000000000000001,
        'sub_row': 0.5,
        'metric': 'mae'}


}

In [0]:
sub = pd.read_csv(f'{config.DATA_DIR}/ano_20000.csv', low_memory=False)

In [0]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, StratifiedKFold

import lightgbm as lgb

score = []

# fit 8(type) model
for idx in type_name:
    ntype = type_name[idx]
    print(ntype)
    x_train_val = pd.read_csv(f'{config.DATA_DIR}/tmp/{ntype}/x_train.csv', index_col=0, low_memory=False)
    print('x_train:', x_train_val.shape)
    x_test_val =pd.read_csv(f'{config.DATA_DIR}/tmp/{ntype}/x_test.csv', index_col=0, low_memory=False)
    print('x_test:', x_test_val.shape)
    ID = pd.read_pickle(f'{config.DATA_DIR}/tmp/{ntype}/ID.csv')
    y_train_val = pd.read_pickle(f'{config.DATA_DIR}/tmp/{ntype}/y_train.csv')

    print(f'------------type{ntype}------------')

    maes = []
    predictions = np.zeros(len(x_test_val))
    preds_train = np.zeros(len(x_train_val))

    n_fold = 5
    folds = StratifiedKFold(n_splits=n_fold, shuffle=False, random_state=42)

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(x_train_val, ID)):
        strLog = "fold {}".format(fold_)
        print(strLog)

        x_tr, x_val = x_train_val.iloc[trn_idx], x_train_val.iloc[val_idx]
        y_tr, y_val = y_train_val.iloc[trn_idx], y_train_val.iloc[val_idx]

        model = lgb.LGBMRegressor(**type_params[idx], n_estimators=30000,random_state=1)
        model.fit(x_tr,
                  y_tr,
                  eval_set=[(x_tr, y_tr), (x_val, y_val)],
                  eval_metric='mae',
                  verbose=1000,
                  early_stopping_rounds=200
                  )

        # predictions
        preds = model.predict(
            x_test_val)  # , num_iteration=model.best_iteration_)
        predictions += preds / folds.n_splits
        preds = model.predict(
            x_train_val)  # , num_iteration=model.best_iteration_)
        preds_train += preds / folds.n_splits

        preds = model.predict(x_val)

        # mean absolute error
        mae = mean_absolute_error(y_val, preds)
        print('MAE: %.6f' % mae)
        print('Score: %.6f' % np.log(mae))
        maes.append(mae)
        print('')

    sub.loc[sub['typei'] == idx, 'scalar_coupling_constant'] = predictions
    score.append(np.mean(maes))
    print(f'{ntype} MAE:', np.mean(maes))
    print(f'{ntype} Score:', np.log(np.mean(maes)))

    print('')

print('')
print('----------------------')
# print('train score:', sum(np.log(score)) / 8)

2_2JHH
x_train: (378036, 114)
x_test: (203126, 114)
------------type2_2JHH------------
fold 0
Training until validation scores don't improve for 200 rounds.
[1000]	training's l1: 0.362777	valid_1's l1: 0.403161
[2000]	training's l1: 0.149648	valid_1's l1: 0.168288
[3000]	training's l1: 0.124327	valid_1's l1: 0.144838
[4000]	training's l1: 0.110142	valid_1's l1: 0.133276
[5000]	training's l1: 0.100335	valid_1's l1: 0.125814
[6000]	training's l1: 0.0927566	valid_1's l1: 0.120381
[7000]	training's l1: 0.0866721	valid_1's l1: 0.116267
[8000]	training's l1: 0.0815827	valid_1's l1: 0.112928
[9000]	training's l1: 0.0772649	valid_1's l1: 0.110228
[10000]	training's l1: 0.0734871	valid_1's l1: 0.107899
[11000]	training's l1: 0.0701705	valid_1's l1: 0.105875
[12000]	training's l1: 0.0671727	valid_1's l1: 0.104121
[13000]	training's l1: 0.0644566	valid_1's l1: 0.102621
[14000]	training's l1: 0.0619746	valid_1's l1: 0.101256
[15000]	training's l1: 0.0596983	valid_1's l1: 0.100029
[16000]	training'

In [0]:
sub.to_csv(f'{config.DATA_DIR}/ano_20000.csv', index=False)

In [0]:
final = sub.drop(['typei'], axis=1)

In [0]:
final.to_csv(f'{config.DATA_DIR}/final.csv', index=False)